# Instalação de Bibliotecas


In [ ]:
!pip install pytelegrambotapi
!pip install unidecode

# Lendo nosso dataset

## Pandas


A biblioteca pandas é uma poderosa biblioteca em Python amplamente utilizada para análise de dados e manipulação de estruturas de dados tabulares, como DataFrames. Ela oferece funcionalidades para carregar, limpar, transformar, analisar e visualizar dados de maneira eficiente.

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/tmelo-uea/cursos/main/dataset.csv', delimiter=';')

In [ ]:
dataset.head()

# Bibliotecas para tratar strings



In [ ]:
import string

# Bibliotecas de NLP

## NLTK
O Natural Language Toolkit (NLTK) é uma biblioteca em Python amplamente utilizada para processamento de linguagem natural (NLP). Ela oferece uma gama de ferramentas que tornam mais fácil a realização de análises textuais e tarefas de NLP, bem como a construção de sistemas de processamento de linguagem natural.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# Biblioteca para remoção de acentuação

In [ ]:
from unidecode import unidecode

# Pré processamento de texto

In [ ]:
def remove_pontuacao(text):
    texto_limpo = ''
    for palavra in text:
      if palavra not in string.punctuation:
        texto_limpo += palavra

    return texto_limpo

In [ ]:
print(string.punctuation)

In [ ]:
def preprocessamento(texto):
    # Remove pontuações e símbolos
    texto = remove_pontuacao(texto)

    # Remover acentos
    texto = unidecode(texto)

    # Converte para minúsculo
    texto = texto.lower()

    # Tokenização
    tokens = word_tokenize(texto)

    # Remover stopwords
    stop_words = stopwords.words('portuguese')
    tokens = [token for token in tokens if token not in stop_words]

    # Juntar as palavras novamente em uma string
    texto_preprocessado = ' '.join(tokens)

    return texto_preprocessado

In [ ]:
dataset["Pergunta_Preprocessada"] = dataset["Pergunta"].apply(preprocessamento)

In [ ]:
dataset.head(n=10)

## Vetorização

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dataset["Pergunta_Preprocessada"])

## Método para obter uma resposta a partir de uma pergunta

In [ ]:
def obter_resposta(pergunta):
    # Pré processa a sentença da pergunta
    pergunta_processada = preprocessamento(pergunta)

    # Transforma a pergunta para um vetor
    pergunta_vector = vectorizer.transform([pergunta_processada])

    # Calcula a similaridade do cosseno
    similaridades = cosine_similarity(pergunta_vector, tfidf_matrix)

    # Obtém o índice da pergunta mais similar
    pergunta_index = similaridades.argmax()

    # Devolve a resposta para o usuário
    return dataset["Resposta"].iloc[pergunta_index]

## Testando Chatbot

In [ ]:
flags = ['Fechar', 'Sair', 'Tchau']
end = False

while not end:
    question = input()
    if question in flags:
        end = True
        print('Finalizando Chat')
        continue

    answer = obter_resposta(question)
    print(answer)

# Integrando com Telegram

In [ ]:
import telebot

API_KEY = '7513630037:AAHNNmZ6g-6t9ODEq0l4OvdPQNhDx-c2a5U'
bot = telebot.TeleBot(API_KEY)

In [ ]:
@bot.message_handler(func=lambda message: True)
def default(message):
    resposta = obter_resposta(message.text)

    bot.reply_to(
        message,
        resposta
    )

In [26]:
bot.polling()

2025-07-11 12:58:51,613 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2025-07-11 12:58:51,616 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1234, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/usr/local/lib/python3.11/dist-packages/telebot/util.py", line 111, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.11/dist-packages/telebot/util.py", line 93, in run
    task(*args, **kwargs)
  